In [1]:
from torch.utils.data import DataLoader
from Dataset.NSRVideoDataset import NSRVideoDataset
from easydict import EasyDict
import torch
from torchvision.transforms import transforms
from torch.utils.tensorboard import SummaryWriter
import time
import os
from Models.VGG16 import VGG16
import torchsummary
from tqdm import tqdm

In [2]:
args = EasyDict({
    'dataset_path' : r"D:\Video-Dataset\2022-NSR-3m", # root directory path
    'split' : (0.2, 0.5), # train/validation, validation/train use train_test_split function
    'dataset_type' : ("Train", "Validation", "Test"),
    'batch_size' : 64,
    'epochs' : 10,
    'learning_rate' : 1e-3,
    'model_name' : "VGG16",
})

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = NSRVideoDataset(args.dataset_path, args.split, args.dataset_type[0], transform)
train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, pin_memory=True, num_workers=1, drop_last=False)

validation_dataset = NSRVideoDataset(args.dataset_path, args.split, args.dataset_type[1], transform)
validation_dataloader = DataLoader(validation_dataset, batch_size=args.batch_size, shuffle=True, pin_memory=True, num_workers=1, drop_last=False)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = VGG16().to(device)
# torchsummary.summary(model, (3, 256, 256))

criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)

In [5]:
EXPERIMENT_DIR = f"./runs/{time.strftime('%Y-%m-%d-%H%M%S')}-{args.model_name}"
os.makedirs(EXPERIMENT_DIR, exist_ok=True)
tensorboard_writer = SummaryWriter(log_dir=EXPERIMENT_DIR)

for epoch in tqdm(range(args.epochs), total=args.epochs, desc="Epoch progress"):
    train_loss, train_accuracy = 0, 0
    validation_loss, validation_accuracy = 0, 0
    total_batch = len(train_dataloader)

    for batch_idx, (datas, labels) in enumerate(tqdm(train_dataloader, total=total_batch, desc="train progress")):
        datas, labels = datas.to(device), labels.to(device)

        predict = model(datas)
        loss = criterion(predict, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item() / len(predict)
        train_accuracy += ((predict > 0.5) == labels).float().sum().item() / len(predict)
    
    print("Epoch: ", "%d" % (epoch + 1), "train_loss: ", "{:.9f}".format(train_loss), "train_accuracy: ", train_accuracy)
    tensorboard_writer.add_scalar("Train Loss", train_loss, epoch + 1)
    tensorboard_writer.add_scalar("Train Accuracy", train_accuracy, epoch + 1)

    with torch.no_grad():
        model.eval()

        for batch_idx, (datas, labels) in enumerate(tqdm(validation_dataloader, total=len(validation_dataloader), desc="validation progress")):
            datas, labels = datas.to(device), labels.to(device)

            predict = model(datas)
            loss = criterion(predict, labels)
            validation_loss += loss.item() / len(predict)
            validation_accuracy += ((predict > 0.5) == labels).float().sum().item() / len(predict)

        print("Epoch: ", "%d" % (epoch + 1), "validation_loss: ", "{:.9f}".format(validation_loss), "validation_accuracy: ", validation_accuracy)
        tensorboard_writer.add_scalar("Validation Loss", validation_loss, epoch + 1)
        tensorboard_writer.add_scalar("Validation Accuracy", validation_accuracy, epoch + 1)


tensorboard_writer.close()

Epoch progress:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
print(datas.size())
print(labels)
print(predict)